## Station Lookup

In [1]:
import pandas as pd

In [2]:
bucket = "sand-test-central-481423469601-us-east-1"
file = "cleaned_historical_trips_2015_2022.csv"

s3_data_location = f"s3://{bucket}/{file}*"
trips = pd.read_csv(s3_data_location, parse_dates = True)
trips.shape

/tmp/ipykernel_628/2246501981.py:5: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  trips = pd.read_csv(s3_data_location, parse_dates = True)


(15276602, 17)

Notes:

- if "Washington Sq" is selected, ID is 402 (previous ID was 451 that expired on 10/15/2020)
    - code here: `max(trips_start[(trips_start["start station name"] == "Washington Sq") & (trips_start["start station id"] == 451)]["starttime"])`
- take out station 572 because it didn't make the cut to be included in the trip stop station model (first trip was past the training date cut-off)

### Trip Start Station

In [3]:
trips_start = trips[(trips["starttime"] > "2020-08-29") & (trips["starttime"] < "2022-09-01")][["starttime", "start station name", "start station id"]]
trips_start.shape

(6059710, 3)

In [4]:
trips_start_lookup = trips_start[["start station name", "start station id"]].drop_duplicates().reset_index(drop = True)
trips_start_lookup.shape

(461, 2)

In [5]:
trips_start_lookup = trips_start_lookup[trips_start_lookup["start station id"] != 451]
trips_start_lookup.shape

(460, 2)

In [6]:
print(trips_start_lookup["start station name"].nunique())
print(sum(pd.isna(trips_start_lookup["start station name"])))
print(trips_start_lookup["start station id"].nunique())
print(sum(pd.isna(trips_start_lookup["start station id"])))

460
0
450
0


In [7]:
#trips_start_lookup.to_csv("trips_start_lookup.csv", index = False)

### Trip Stop Station

In [21]:
trips_stop = trips[(trips["stoptime"] > "2020-08-29") & (trips["stoptime"] < "2022-09-01")][["stoptime", "end station name", "end station id"]]
trips_stop.shape

(6059354, 3)

In [22]:
trips_stop_lookup = trips_stop[["end station name", "end station id"]].drop_duplicates().reset_index(drop = True)
trips_stop_lookup.shape

(462, 2)

In [23]:
trips_stop_lookup = trips_stop_lookup[trips_stop_lookup["end station id"] != 451]
trips_stop_lookup.shape

(461, 2)

In [24]:
print(trips_stop_lookup["end station name"].nunique())
print(sum(pd.isna(trips_stop_lookup["end station name"])))
print(trips_stop_lookup["end station id"].nunique())
print(sum(pd.isna(trips_stop_lookup["end station id"])))

461
0
451
0


In [13]:
#trips_stop_lookup.to_csv("trips_stop_lookup.csv", index = False)

### Combine

In [25]:
trips_start_lookup.columns = ["station name", "station id"]

In [26]:
trips_stop_lookup.columns = ["station name", "station id"]

In [27]:
trips_lookup = pd.concat([trips_start_lookup, trips_stop_lookup], ignore_index = True, axis = 0).drop_duplicates().reset_index(drop = True)
trips_lookup.shape

(461, 2)

In [28]:
trips_lookup = trips_lookup[trips_lookup["station id"] != 572]
trips_lookup.shape

(460, 2)

In [29]:
trips_lookup.to_csv("trips_lookup.csv", index = False)